# Creating and modelling metallic supercells

In this section we will be concerned with modelling supercells of aluminium.
When dealing with periodic problems there is no unique definition of the
lattice: Clearly any duplication of the lattice along an axis is also a valid
repetitive unit to describe exactly the same system.
This is exactly what a **supercell** is: An $n$-fold repetition along one of the
axes of the original lattice.

The following code achieves this for aluminium:

In [1]:
using DFTK
using LinearAlgebra
using ASEconvert

function aluminium_setup(repeat=1; Ecut=7.0, kgrid=[2, 2, 2])
    a = 7.65339
    lattice = a * Matrix(I, 3, 3)
    Al = ElementPsp(:Al, psp=load_psp("hgh/lda/al-q3"))
    atoms     = [Al, Al, Al, Al]
    positions = [[0.0, 0.0, 0.0], [0.0, 0.5, 0.5], [0.5, 0.0, 0.5], [0.5, 0.5, 0.0]]
    unit_cell = periodic_system(lattice, atoms, positions)

    # Make supercell in ASE:
    # We convert our lattice to the conventions used in ASE, make the supercell
    # and then convert back ...
    supercell_ase = convert_ase(unit_cell) * pytuple((repeat, 1, 1))
    supercell     = pyconvert(AbstractSystem, supercell_ase)

    # Unfortunately right now the conversion to ASE drops the pseudopotential information,
    # so we need to reattach it:
    supercell = attach_psp(supercell, Al="hgh/lda/al-q3")

    # Construct an LDA model and discretise
    # Note: We disable symmetries explicitly here. Otherwise the problem sizes
    #       we are able to run on the CI are too simple to observe the numerical
    #       instabilities we want to trigger here.
    model = model_LDA(supercell; temperature=1e-3, symmetries=false)
    PlaneWaveBasis(model; Ecut, kgrid)
end;

As part of the code we are using a routine inside the ASE,
the [atomistic simulation environment](https://wiki.fysik.dtu.dk/ase/index.html)
for creating the supercell and make use of the two-way interoperability of
DFTK and ASE. For more details on this aspect see the documentation
on Input and output formats.

Write an example supercell structure to a file to plot it:

In [2]:
setup = aluminium_setup(5)
convert_ase(periodic_system(setup.model)).write("al_supercell.png")

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/neSFl/src/ASEconvert.jl:123
┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/neSFl/src/ASEconvert.jl:123


Python None

<img src="https://docs.dftk.org/stable/examples/al_supercell.png" width=500 height=500 />

As we will see in this notebook the modelling of a system generally becomes
harder if the system becomes larger.

- This sounds like a trivial statement as *per se* the cost per SCF step increases
  as the system (and thus $N$) gets larger.
- But there is more to it:
  If one is not careful also the *number of SCF iterations* increases
  as the system gets larger.
- The aim of a proper computational treatment of such supercells is therefore
  to ensure that the **number of SCF iterations remains constant** when the
  system size increases.

For achieving the latter DFTK by default employs the `LdosMixing`
preconditioner [^HL2021] during the SCF iterations. This mixing approach is
completely parameter free, but still automatically adapts to the treated
system in order to efficiently prevent charge sloshing. As a result,
modelling aluminium slabs indeed takes roughly the same number of SCF iterations
irrespective of the supercell size:

[^HL2021]:
   M. F. Herbst and A. Levitt.
   *Black-box inhomogeneous preconditioning for self-consistent field iterations in density functional theory.*
   J. Phys. Cond. Matt *33* 085503 (2021). [ArXiv:2009.01665](https://arxiv.org/abs/2009.01665)

In [3]:
self_consistent_field(aluminium_setup(1); tol=1e-4);

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/neSFl/src/ASEconvert.jl:123
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -8.298312634392                   -0.85    5.0         
  2   -8.300166242283       -2.73       -1.26    1.2    105ms
  3   -8.300425969349       -3.59       -1.88    2.1    119ms
  4   -8.300462157094       -4.44       -2.74    2.6    130ms
  5   -8.300464065712       -5.72       -3.07    2.6    180ms
  6   -8.300464354302       -6.54       -3.25   10.2    222ms
  7   -8.300464503789       -6.83       -3.40    1.6    118ms
  8   -8.300464583492       -7.10       -3.56    3.0    139ms
  9   -8.300464626543       -7.37       -3.76    2.9    161ms
 10   -8.300464637052       -7.98       -3.91    1.8    140ms
 11   -8.300464643473       -8.19       -4.25    3.0    207ms


In [4]:
self_consistent_field(aluminium_setup(2); tol=1e-4);

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/neSFl/src/ASEconvert.jl:123
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -16.67369816663                   -0.70    5.8         
  2   -16.67790526757       -2.38       -1.14    1.8    293ms
  3   -16.67905545368       -2.94       -1.86    2.9    358ms
  4   -16.67917388896       -3.93       -2.56    4.1    368ms
  5   -16.67927727953       -3.99       -3.09    4.2    403ms
  6   -16.67928600484       -5.06       -3.51    2.6    346ms
  7   -16.67928621728       -6.67       -3.87    2.8    330ms
  8   -16.67928622144       -8.38       -4.59    1.8    275ms


In [5]:
self_consistent_field(aluminium_setup(4); tol=1e-4);

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/neSFl/src/ASEconvert.jl:123
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -33.32702375359                   -0.56    6.9         
  2   -33.33436931658       -2.13       -1.00    1.5    977ms
  3   -33.33593849098       -2.80       -1.72    5.4    1.30s
  4   -33.33617279000       -3.63       -2.63    4.9    1.25s
  5   -33.33692025371       -3.13       -2.66    8.1    1.63s
  6   -33.33693601179       -4.80       -2.93    3.0    1.04s
  7   -33.33694350332       -5.13       -3.66    2.9    1.18s
  8   -33.33694375748       -6.59       -4.09    4.4    1.35s


When switching off explicitly the `LdosMixing`, by selecting `mixing=SimpleMixing()`,
the performance of number of required SCF steps starts to increase as we increase
the size of the modelled problem:

In [6]:
self_consistent_field(aluminium_setup(1); tol=1e-4, mixing=SimpleMixing());

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/neSFl/src/ASEconvert.jl:123
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -8.298492749671                   -0.85    4.9         
  2   -8.300274388325       -2.75       -1.59    1.0   99.3ms
  3   -8.300433809164       -3.80       -2.63    3.0    112ms
  4   -8.300415060520   +   -4.73       -2.40    7.4    194ms
  5   -8.300464257301       -4.31       -3.35    1.0   88.0ms
  6   -8.300464585210       -6.48       -3.79    4.2    153ms
  7   -8.300464641344       -7.25       -4.31    1.5    110ms


In [7]:
self_consistent_field(aluminium_setup(4); tol=1e-4, mixing=SimpleMixing());

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/neSFl/src/ASEconvert.jl:123
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -33.32411906684                   -0.56    7.1         
  2   -33.33205968282       -2.10       -1.28    1.0    988ms
  3   -32.80087755683   +   -0.27       -1.20    3.9    1.30s
  4   -31.73058686615   +    0.03       -0.96    5.4    1.79s
  5   -33.31633781169        0.20       -1.68    4.2    1.24s
  6   -33.32845306444       -1.92       -1.93    2.4    1.08s
  7   -33.13981392821   +   -0.72       -1.41    4.6    1.33s
  8   -33.33601663666       -0.71       -2.48    3.6    1.16s
  9   -33.33674168873       -3.14       -2.59    3.5    1.33s
 10   -33.33663183436   +   -3.96       -2.50    2.2    1.08s
 11   -33.33691626650       -3.55       -3.25    2.0    972ms
 12   -33.33693881532       -4.65      

For completion let us note that the more traditional `mixing=KerkerMixing()`
approach would also help in this particular setting to obtain a constant
number of SCF iterations for an increasing system size (try it!). In contrast
to `LdosMixing`, however, `KerkerMixing` is only suitable to model bulk metallic
system (like the case we are considering here). When modelling metallic surfaces
or mixtures of metals and insulators, `KerkerMixing` fails, while `LdosMixing`
still works well. See the Modelling a gallium arsenide surface example
or [^HL2021] for details. Due to the general applicability of `LdosMixing` this
method is the default mixing approach in DFTK.